# Mensajes Twilio

In [40]:
import os 
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, PHONE_NUMBER, API_KEY_WAPI
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

from datetime import datetime

### Creacion de la URL

In [41]:
query = "Madrid"
api_key = API_KEY_WAPI
url_tiempo = "http://api.weatherapi.com/v1/forecast.json?key=a4a2457d58b2444e8ee171445231707&q=Madrid&days=1&aqi=no&alerts=no"

### Creación del JSON

In [42]:
response = requests.get(url_tiempo).json()
response

{'location': {'name': 'Madrid',
  'region': 'Madrid',
  'country': 'Spain',
  'lat': 40.4,
  'lon': -3.68,
  'tz_id': 'Europe/Madrid',
  'localtime_epoch': 1689700955,
  'localtime': '2023-07-18 19:22'},
 'current': {'last_updated_epoch': 1689700500,
  'last_updated': '2023-07-18 19:15',
  'temp_c': 36.0,
  'temp_f': 96.8,
  'is_day': 1,
  'condition': {'text': 'Sunny',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png',
   'code': 1000},
  'wind_mph': 10.5,
  'wind_kph': 16.9,
  'wind_degree': 210,
  'wind_dir': 'SSW',
  'pressure_mb': 1019.0,
  'pressure_in': 30.09,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 21,
  'cloud': 0,
  'feelslike_c': 34.4,
  'feelslike_f': 94.0,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 9.0,
  'gust_mph': 18.6,
  'gust_kph': 29.9},
 'forecast': {'forecastday': [{'date': '2023-07-18',
    'date_epoch': 1689638400,
    'day': {'maxtemp_c': 41.8,
     'maxtemp_f': 107.2,
     'mintemp_c': 24.3,
     'mintemp_f': 75.7,
     'avgtemp_c': 3

### Para tener mejor visualización del JSON

In [43]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [44]:
response["forecast"]["forecastday"][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

### Obtención de las 24 hrs

In [45]:
len(response["forecast"]["forecastday"][0]["hour"])

24

### Obtención de la fecha

In [46]:
response["forecast"]["forecastday"][0]["hour"][1]["time"].split()[0]

'2023-07-18'

### Transfomación de la hora en un Entero

In [47]:
int(response["forecast"]["forecastday"][0]["hour"][1]["time"].split()[1].split(":")[0])

1

### Condición del día

In [48]:
response["forecast"]["forecastday"][0]["hour"][0]["condition"]["text"]

'Clear'

### Temperatura en Grados Centigrados

In [49]:
response["forecast"]["forecastday"][0]["hour"][0]["temp_c"]

29.5

### Si llovera o no (0 = no llovera; 1 = si llovera)

In [50]:
response["forecast"]["forecastday"][0]["hour"][0]["will_it_rain"]

0

### Probabilidad de que llovera en porcentaje

In [51]:
response["forecast"]["forecastday"][0]["hour"][0]["chance_of_rain"]

0

# Dataframe

In [52]:
def get_forecast(response, i):
    fecha = response["forecast"]["forecastday"][0]["hour"][i]["time"].split()[0]
    hora = int(response["forecast"]["forecastday"][0]["hour"][i]["time"].split()[1].split(":")[0])
    condicion = response["forecast"]["forecastday"][0]["hour"][i]["condition"]["text"]
    tempe = response["forecast"]["forecastday"][0]["hour"][i]["temp_c"]
    rain = response["forecast"]["forecastday"][0]["hour"][i]["will_it_rain"]
    prob_rain = response["forecast"]["forecastday"][0]["hour"][i]["chance_of_rain"]

    return fecha, hora , condicion, tempe, rain, prob_rain


In [53]:
datos = []
for i in tqdm(range(len(response["forecast"]["forecastday"][0]["hour"])), colour = "green"):
    datos.append(get_forecast(response, i))

100%|██████████| 24/24 [00:00<?, ?it/s]


In [54]:
datos[0]

('2023-07-18', 0, 'Clear', 29.5, 0, 0)

In [55]:
col = ["Fecha", "Hora", "Condicion", "Temperatura", "Lluvia", "Probabilidad_Lluvia"]
df = pd.DataFrame(datos, columns = col)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,Probabilidad_Lluvia
0,2023-07-18,0,Clear,29.5,0,0
1,2023-07-18,1,Clear,28.0,0,0
2,2023-07-18,2,Clear,26.8,0,0
3,2023-07-18,3,Clear,25.9,0,0
4,2023-07-18,4,Clear,25.1,0,0
5,2023-07-18,5,Clear,24.3,0,0
6,2023-07-18,6,Clear,25.2,0,0
7,2023-07-18,7,Partly cloudy,27.5,0,0
8,2023-07-18,8,Partly cloudy,29.5,0,0
9,2023-07-18,9,Partly cloudy,31.7,0,0


### Condición para mostrar en el msj solo las horas de una posible lluvia entre las 8 am y 22 pm

In [56]:
df_rain = df[(df["Lluvia"] == 0) & (df["Hora"] > 8) & (df["Hora"] < 22)] #Se coloco 0 para realizar la prueba de manera optima, pero es un 1 de la manera correcta
df_rain =  df_rain[["Hora", "Condicion"]]
df_rain.set_index("Hora", inplace = True)

In [57]:
df_rain

,Condicion
Hora,
9,Partly cloudy
10,Sunny
11,Sunny
12,Sunny
13,Sunny
14,Sunny
15,Sunny
16,Sunny
17,Sunny


### Se crea un template para el mensaje

In [58]:
"\n Hola!!! Equipo TSOFT España \n\n\n El pronostico del tiempo para hoy es" + df["Fecha"][0] + "en" + query + "es : \n\n\n" + str(df_rain) 

'\n Hola!!! Equipo TSOFT España \n\n\n El pronostico del tiempo para hoy es2023-07-18enMadrides : \n\n\n          Condicion\nHora               \n9     Partly cloudy\n10            Sunny\n11            Sunny\n12            Sunny\n13            Sunny\n14            Sunny\n15            Sunny\n16            Sunny\n17            Sunny\n18            Sunny\n19            Sunny\n20            Sunny\n21            Sunny'

In [59]:
PHONE_NUMBER

19897955812

### Mensaje SMS desde TWILIO

In [60]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages.create(
                    body= "\nHola!!! Equipo TSOFT España \n\n\nEl pronostico del tiempo para hoy " + df["Fecha"][0] + " en " + query + " es : \n\n\n" + str(df_rain), 
                    from_ = PHONE_NUMBER,
                    to = "+525513979927"
                )
print("Mensaje Enviado " + message.sid)

Mensaje Enviado SMfc2dbc82f524c40496915aa05bb13e2d
